# Build simple agents that perform navigation on the line and the loop

Agents we want to study
<ul stlye="list-style: square">
    <li> Go inwards </li>
    <li> Simple Q-learning agent </li>
    <li> Model based agent </li>
    <li> Go up and down stereotyped </li>
    <li> Fully random decisions </li>
<ul>

In [21]:
%matplotlib inline

import numpy as np
import os
import sys
import re
import matplotlib.pyplot as plt
import scipy.stats as stt
import pandas as pd
from datetime import datetime
from functools import partial
import matplotlib
import networkx as nx
import seaborn
from scipy.ndimage import gaussian_filter1d

seaborn.set(font_scale=1.5,style='ticks')

In [180]:
class base_agent(object):
    """
    This is a generic class that other agents (with different policies)
    inherit from. This class implements the basic task_logic
    """
    def __init__(self,learning_params=None,task_params=None):
        """ 
        For simplicity, in the first instance, will assume that
        
        Arguments:
        =======================
        
        task_params (dict):  dictionary containing parameters that are required for running of the task
        
        learning_params (dict): parameters necessary for specifying and updating the policy
        
        """
        
        if task_params is None:
            task_params = {'len_graph':9,
                           'graph_type':'line',
                           'rew_locs_session':list(range(9)),
                           'rewards_pre_switch': 20,
                           'reward_switch_p': 0.2,
                            }

        
        self.learning_params = learning_params
        self.task_params = task_params
        
        
        #helper params
        self._all_states = list(range(self.task_params['len_graph']))
        

        self._init_task_variables()
        self._init_stores()
        

        
    def _init_task_variables(self):
        #initialise variables
        self.forced = False
        self.reward_location = None
        self.current_state = None
        self.current_port = None
        self.current_reward_location = None
        self.available_states = []
        self.choices_since_reward = 0
        self.n_rewards_at_loc = 0
        self.n_reward_total = 0
        
    def _init_stores(self):
        #this is the information to extract at the end to feed into
        #the standard analysis pipeline
        self.state_seq = []
        self.rew_list = []
        self.port_seq = []
        self.forced_seq = []

    def set_available_states(self):
        """ updates available_states variables"""
        
        
        #if bottom edge
        if self.current_state==0:
            if self.task_params['graph_type']=='line':
                self.available_states = [1]
            else:
                self.available_states = [self.task_params['len_graph']-1,1]
                
        #elif top edge
        elif self.current_state==(self.task_params['len_graph']-1):
            
            if self.task_params['graph_type']=='line':
                self.available_states = [self.task_params['len_graph']-2]
            else:
                self.available_states = [self.task_params['len_graph']-2,0]
                
        #with states either side
        else:
            self.available_states = [self.current_state-1,self.current_state+1]

    
    def init_trial(self):
        """ Initialise the task"""
        if ((self.reward_location is None) or 
            (self.n_rewards_at_loc>self.task_params['rewards_pre_switch'] and
                np.random.uniform(0,1)>self.task_params['reward_switch_p'])):
            self.update_reward_location()
        
            

        self.current_state = np.random.choice(self.poss_starting_states)
        self.set_available_states()
        #print(self.curr)
        #print(self.available_states)
        self.state_seq.append(self.current_state)
        self.port_seq.append(self.current_port)
        self.rew_list.append(False)

        
    def run(self,n_trials=3e3):
        """ Run behaviour """
        self.init_trial()
        #if self.task_params
        #print(n_trials)
        rew = False
        while self.n_reward_total<n_trials:
            if rew:
                self.init_trial()

            print(self.available_states)
            rew = self.update_state()
            
            self.state_seq.append(self.current_state)
            self.port_seq.append(self.current_port)
            self.rew_list.append(rew)
                    
        
    
    def update_state(self):
        """ Received choice of 0 or 1"""
        self.do_policy()
        self.set_available_states()

        rew = False
        if self.current_state==self.reward_location:
            self.choices_since_reward = 0
            self.n_rewards_at_loc += 1
            self.choices_since_reward =0 
            self.n_reward_total += 1
            self.rew_list.append(True)
            rew = True
        else:
            self.choices_since_reward += 1

        return rew
            
    
    def update_reward_location(self):
        self.reward_location = np.random.choice(self.task_params['rew_locs_session'])
        self.poss_starting_states = [i for i in self.task_params['rew_locs_session'] if i!=self.reward_location]

    
    
    
    def do_policy(self):
        self.current_state = np.random.choice(self.available_states)

In [181]:
aa.current_state

2

In [182]:
aa.available_states

[1, 3]

In [183]:
aa.set_available_states()

In [184]:
aa.reward_location

3

In [185]:
aa = base_agent()

In [187]:
aa.run(n_trials=1)

1
[0, 2]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1, 3]
[2, 4]
[3, 5]
[2, 4]
[3, 5]
[4, 6]
[3, 5]
[2, 4]
[1, 3]
[0, 2]
[1, 3]
[2, 4]
[1, 3]
[2, 4]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1, 3]
[2, 4]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1, 3]
[0, 2]
[1, 3]
[2, 4]
[1, 3]
[0, 2]
[1, 3]
[0, 2]
[1]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1, 3]
[0, 2]
[1, 3]
[2, 4]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[0, 2]
[1]
[0, 2]
[1, 3]
[2, 4]
[1, 3]
[2, 4]
[1, 3]
[2, 4]
[3, 5]
[4, 6]
[5, 7]
[6, 8]
[5, 7]
[6, 8]


In [110]:
aa.available_states

In [112]:
aa.current_state

5

In [ ]:
class diffusion_agent(base_agent):
    
    def __init__(self,learning_params,task_params):
        pass

In [18]:
class towards_middle_agent(base_agent):
    
    def __init__(self,learning_params,task_params):
        pass

In [19]:
class Qlearner(base_agent):
    
    def __init__(self,learning_params,task_params):
        pass

In [20]:
class Model_based_agent(base_agent):
    
    def __init__(self,learning_params,task_params):
        pass